In [1]:
import pandas as pd

If you use this dataset, please cite:

Јapczyсski M., Biaіow№s S. (2013) Discovering Patterns of Users' Behaviour in an E-shop - 
Comparison of Consumer Buying Behaviours in Poland and Other European Countries, 
“Studia Ekonomiczne”, nr 151, “La sociйtй de l'information : perspective europйenne et 
globale : les usages et les risques d'Internet pour les citoyens et les consommateurs”, p. 144-
153.


In [2]:
df = pd.read_csv('data/e-shop_clothing_2008.csv', sep=';')

In [3]:
df.head()

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   year                     165474 non-null  int64 
 1   month                    165474 non-null  int64 
 2   day                      165474 non-null  int64 
 3   order                    165474 non-null  int64 
 4   country                  165474 non-null  int64 
 5   session ID               165474 non-null  int64 
 6   page 1 (main category)   165474 non-null  int64 
 7   page 2 (clothing model)  165474 non-null  object
 8   colour                   165474 non-null  int64 
 9   location                 165474 non-null  int64 
 10  model photography        165474 non-null  int64 
 11  price                    165474 non-null  int64 
 12  price 2                  165474 non-null  int64 
 13  page                     165474 non-null  int64 
dtypes: int64(13), object

Переименуем столбцы для лучшей информативности

In [5]:
df = df.rename(columns={
    'session ID': 'session_id',
    'page 1 (main category)': 'main_category',
    'page 2 (clothing model)': 'clothing_model',
    'model photography': 'model_photography',
    'price 1': 'price', 
    'price 2': 'is_price_higher_avg',
})

In [6]:
df.isnull().sum()

year                   0
month                  0
day                    0
order                  0
country                0
session_id             0
main_category          0
clothing_model         0
colour                 0
location               0
model_photography      0
price                  0
is_price_higher_avg    0
page                   0
dtype: int64

Так как пропуски отсутствуют, заполнять ничего не нужно
Перекодируем категориальный признак

page 2 (clothing model)

df.clothing_model.values эквивалентно df['clothing_model.values']

In [7]:
len(set(df.clothing_model.values))

217

Он имеет 217 различных значений, поэтому простым mapping не следует перекодировать

In [8]:
df = pd.get_dummies(df, prefix=['cm'], columns=['clothing_model'])

перекодировать порядковые признаки

Значения месяца лежат в порядке 4..8 - перекодируем их в значения 0..4

YEAR имеет ровно одно значение - перекодируем его в 0

Перекодируем значения дня, страны, id сессии, заказа, расположения нажатия мышки, главной категории, модели при фотографировании, страницы в (0, ...) вместо (1, ...)

In [9]:
df.year = df.year - df.year.min()
df.month = df.month - df.month.min()
df.day = df.day - df.day.min()
df.session_id = df.session_id - df.session_id.min()
df.order = df.order - df.order.min()
df.location = df.location - df.location.min()
df.country = df.country - df.country.min()
df.main_category = df.main_category - df.main_category.min()
df.colour = df.colour - df.colour.min()
df.model_photography = df.model_photography - df.model_photography.min()
df.page = df.page - df.page.min()

привести бинарные признаки к 1 или 0 - таких не обнаружено

однако, присутствует признак price 2, который принимает всего 2 значения - 1 (true) и 2 (false). Перекодируем егои переименуем, для большей информативности

In [10]:
# Возможно упрощенное форматирование - вычитание 1
# Однако тогда True будет соответствовать 0, а False - 1, что семантически неверно
mapping = {1 : 1, 2 : 0}
df.is_price_higher_avg = df.is_price_higher_avg.map(mapping)
print(df.is_price_higher_avg)

0         0
1         0
2         1
3         0
4         1
         ..
165469    1
165470    1
165471    0
165472    1
165473    1
Name: is_price_higher_avg, Length: 165474, dtype: int64


Посмотрим на получившийся датасет

In [11]:
df

,year,month,day,order,country,session_id,main_category,colour,location,model_photography,...,cm_P75,cm_P76,cm_P77,cm_P78,cm_P79,cm_P8,cm_P80,cm_P81,cm_P82,cm_P9
0,0,0,0,0,28,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,28,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,2,28,0,1,9,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,3,28,0,1,5,5,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,4,28,0,1,3,2,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165469,0,4,12,0,28,24023,1,1,3,0,...,0,0,0,0,0,0,0,0,0,0
165470,0,4,12,0,8,24024,0,2,3,0,...,0,0,0,0,0,0,0,0,0,0
165471,0,4,12,0,33,24025,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
165472,0,4,12,1,33,24025,2,11,0,0,...,0,0,0,0,0,0,0,0,0,0


Покажем, что все значения - целочисленные неотрицательные

In [12]:
print(df.min().min())
print(df.drop('price', axis=1).min().max())

0
0


### Значения price решено не приводить к (0, ...), поскольку теряется информативность данной колонки

сохранить отредактированный датасет рядом со старым в формате csv: <название датасета>_preprocessed.csv

In [13]:
df.to_csv('data/e-shop_clothing_2008_preprocessed.csv')